In [ ]:
import pandas as pd
import os
import numpy as np
import re

pd.set_option("display.max_columns", 999)

In [ ]:
os.getcwd()

In [ ]:
os.chdir('/Users/hayley/Documents/p4ds/patent_search')

In [ ]:
# sample_data = pd.read_csv('data_preprocess/data.csv')

In [ ]:
# sample_data.to_excel("data_preprocess/data.xlsx")

In [ ]:
# sample_data = pd.read_excel('data_preprocess/data.xlsx')

In [ ]:
# for col in sample_data.columns:
#     sample_data[col] = sample_data[col].str.replace('\n', '')

In [ ]:
# sample_data.to_excel("data_preprocess/data.xlsx")

# Preprocessing
- 1) problem with extracting reference (number) from texts -> some reference codes are not just numbers. it's number + alphabet e.g. 202a, 202b, or sometimes just uppercase alphabets e.g. T, SR~SZ
- 2) so I tried to first extract the reference codes from "부호에 대한 설명" column. (Because then I can find those specific reference codes inside of texts.) But extracting reference codes from "부호에 대한 설명" was difficult because currently all \<code\>: \<description\> string pairs are concatenated without \n and the parsing reference codes by just relying on regular expression was almost impossible. (some reference codes were uppercase alphabets, but including uppercase alphabets for codes caused problems.) So I asked Mooho if he could leave \n characters left for "부호에 대한 설명" section. He said yes, and I paused developing regex rules further.

In [ ]:
# sample_data = pd.read_csv('pdf_process/data.csv', index_col=0)
# sample_data.to_excel('pdf_process/data.xlsx')
sample_data = pd.read_excel('pdf_process/data.xlsx')

In [ ]:
def extract_ref(x):
    
    init_extract = set(re.findall(r"\(\d*[a-zA-Z]?\)|도 \d+", str(x)))
    if len(init_extract) > 0:
        return init_extract
    else:
        return np.nan

In [ ]:
select_columns = ['요약', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
                 '발명의효과', '발명을실시하기위한구체적인내용']

In [ ]:
for col in select_columns:
    sample_data[f"{col}_ref"] = sample_data[col].apply(extract_ref)

In [ ]:
def extract_description_for_image(x):
    descs = re.findall(r'도 ?\d+[의은는][^\.]+\.', x)
    
    return dict(zip([f'도 {i}' for i in range(1, len(descs)+1)], descs))

sample_data['도면의간단한설명_dict'] = sample_data['도면의간단한설명'].apply(extract_description_for_image)

In [ ]:
def extract_description_for_code(x):
    ids = re.findall(r'([\d]+)[a-zA-Z]?,? ?[\da-zA-Z]*(?= ?[:;]+)', str(x)) # became ugly to take care of cases like '311b, 311c'
    ids = [x.strip() for x in ids]
    descs = re.findall(r'(?<=[:;])[\-0-9ㄱ-ㅎ가-힣a-zA-Z ]+', str(x))
    descs = [re.sub(r'([\d]+)[a-zA-Z]?$', '', x.strip()) for x in descs]
    descs = [x.strip() for x in descs]
    if len(descs)>0: # handling noise from data parsing ('도면' got attached to '부호의설명' column)
        descs[-1] = re.sub('도면.+', '', descs[-1]).strip()
        
    return_dict = dict(zip(ids, descs))
    
    initial_keys = list(return_dict.keys()) # if there're keys like '311b, 311c', split the key into two different keys and add the same value.
    for key in initial_keys:
        if ',' in key:
            value = return_dict[key]
            return_dict.pop(key)
            subkey_list = key.split(',')
            for subkey in subkey_list:
                return_dict[subkey.strip()] = value
    
    return return_dict

In [ ]:
sample_data.loc[:,'부호의설명_dict'] = sample_data['부호의설명'].apply(extract_description_for_code)

In [81]:
print(pytesseract.image_to_string(Image.open('data_preprocess/image/pdf_1020177009557/p11-53.png')))

# OCR

- Tesseract : 잘 못함
- 다른 오픈소스 : pyocr -> tesseract랑 똑같음 / calamari-ocr -> tensorflow 설치해야 되고 등등 maintain이 잘 안되는 패키지 인듯
- GCP vision api : 일단 ui 에서 테스트 해보고 copy json output 버튼 눌러가지고 json output 복사해와서 json output에서 텍스트만 파싱하는 것만 짬

In [ ]:
import pytesseract
from PIL import Image

## tesseract를 먼저 다운받고, 다운받은 경로를 넣어주어야 함.
## You need to first download tesseract & insert the path to the exe file below.
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.3.3/bin/tesseract'

In [47]:
# text image to string
from glob import glob

image_paths = glob('data_preprocess/image/*/*.png')

for img in image_paths:
    print(img)
    print(pytesseract.image_to_string(Image.open(img), lang='eng'))

data_preprocess/image/pdf_1020120156759/p8-6.png

data_preprocess/image/pdf_1020120156759/p7-31.png
A256
ART
129
A120
A158
123,
121

133

A35
437

10 130110 145: 147
140- 143


data_preprocess/image/pdf_1020120156759/p7-30.png
110 5 110

1 ~_-f


data_preprocess/image/pdf_1020120156759/p0-6.png

data_preprocess/image/pdf_1020120156759/p9-39.png
200

212
214a

222 224a 226 228

data_preprocess/image/pdf_1020120156759/p8-35.png
200

300

320
310


data_preprocess/image/pdf_1020180014052/p14-56.png

data_preprocess/image/pdf_1020180014052/p12-6.png

data_preprocess/image/pdf_1020180014052/p15-60.png

data_preprocess/image/pdf_1020180014052/p0-6.png

data_preprocess/image/pdf_1020180014052/p13-52.png

data_preprocess/image/pdf_1020180014052/p11-43.png

data_preprocess/image/pdf_1020180014052/p11-44.png

data_preprocess/image/pdf_1020180014052/p11-45.png

data_preprocess/image/pdf_1020177009557/p13-66.png
1

4

5

\


data_preprocess/image/pdf_1020177009557/p13-67.png
La

4 a, 3 i
cs ot “
Q

In [71]:
for i in json_output['fullTextAnnotation']['pages'][0]['blocks']:
    for j in i['paragraphs']:
        for k in j['words']:
            curr_word = ''
            for l in k['symbols']:
                curr_word += l['text']
            if curr_word.startswith('-'):
                curr_word = curr_word[1:]
            print(curr_word)

30
100
40
20
10


In [76]:
annotations = json_output['textAnnotations'][0]['description'].split('\n')
print(annotations)

['-30', '100', '40', '20', '-10']


In [78]:
annotations = json_output['textAnnotations'][0]['description'].split('\n')
print(annotations)

['15', '***************', '17']


In [77]:
json_output = {
  "cropHintsAnnotation": {
    "cropHints": [
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 510
            },
            {
              "x": 1155
            },
            {
              "x": 1155,
              "y": 805
            },
            {
              "x": 510,
              "y": 805
            }
          ]
        },
        "confidence": 0.5625001,
        "importanceFraction": 0.67390424
      },
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 291
            },
            {
              "x": 1097
            },
            {
              "x": 1097,
              "y": 805
            },
            {
              "x": 291,
              "y": 805
            }
          ]
        },
        "confidence": 0.5625,
        "importanceFraction": 0.84238034
      },
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 218
            },
            {
              "x": 1185
            },
            {
              "x": 1185,
              "y": 805
            },
            {
              "x": 218,
              "y": 805
            }
          ]
        },
        "confidence": 0.55645883,
        "importanceFraction": 1
      }
    ]
  },
  "fullTextAnnotation": {
    "pages": [
      {
        "blocks": [
          {
            "blockType": "TEXT",
            "boundingBox": {
              "vertices": [
                {
                  "x": 44,
                  "y": 493
                },
                {
                  "x": 86,
                  "y": 493
                },
                {
                  "x": 86,
                  "y": 522
                },
                {
                  "x": 44,
                  "y": 522
                }
              ]
            },
            "confidence": 0.98961794,
            "paragraphs": [
              {
                "boundingBox": {
                  "vertices": [
                    {
                      "x": 44,
                      "y": 493
                    },
                    {
                      "x": 86,
                      "y": 493
                    },
                    {
                      "x": 86,
                      "y": 522
                    },
                    {
                      "x": 44,
                      "y": 522
                    }
                  ]
                },
                "confidence": 0.98961794,
                "words": [
                  {
                    "boundingBox": {
                      "vertices": [
                        {
                          "x": 44,
                          "y": 493
                        },
                        {
                          "x": 86,
                          "y": 493
                        },
                        {
                          "x": 86,
                          "y": 522
                        },
                        {
                          "x": 44,
                          "y": 522
                        }
                      ]
                    },
                    "confidence": 0.98961794,
                    "symbols": [
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 44,
                              "y": 494
                            },
                            {
                              "x": 62,
                              "y": 494
                            },
                            {
                              "x": 62,
                              "y": 522
                            },
                            {
                              "x": 44,
                              "y": 522
                            }
                          ]
                        },
                        "confidence": 0.98852473,
                        "text": "1"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 64,
                              "y": 493
                            },
                            {
                              "x": 86,
                              "y": 493
                            },
                            {
                              "x": 86,
                              "y": 521
                            },
                            {
                              "x": 64,
                              "y": 521
                            }
                          ]
                        },
                        "confidence": 0.9907112,
                        "property": {
                          "detectedBreak": {
                            "type": "LINE_BREAK"
                          }
                        },
                        "text": "5"
                      }
                    ]
                  }
                ]
              }
            ]
          },
          {
            "blockType": "TEXT",
            "boundingBox": {
              "vertices": [
                {
                  "x": 475,
                  "y": 504
                },
                {
                  "x": 102,
                  "y": 504
                },
                {
                  "x": 102,
                  "y": 492
                },
                {
                  "x": 475,
                  "y": 492
                }
              ]
            },
            "confidence": 0.33337253,
            "paragraphs": [
              {
                "boundingBox": {
                  "vertices": [
                    {
                      "x": 475,
                      "y": 504
                    },
                    {
                      "x": 102,
                      "y": 504
                    },
                    {
                      "x": 102,
                      "y": 492
                    },
                    {
                      "x": 475,
                      "y": 492
                    }
                  ]
                },
                "confidence": 0.33337253,
                "words": [
                  {
                    "boundingBox": {
                      "vertices": [
                        {
                          "x": 475,
                          "y": 504
                        },
                        {
                          "x": 102,
                          "y": 504
                        },
                        {
                          "x": 102,
                          "y": 492
                        },
                        {
                          "x": 475,
                          "y": 492
                        }
                      ]
                    },
                    "confidence": 0.33337253,
                    "symbols": [
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 475,
                              "y": 504
                            },
                            {
                              "x": 465,
                              "y": 504
                            },
                            {
                              "x": 465,
                              "y": 493
                            },
                            {
                              "x": 475,
                              "y": 493
                            }
                          ]
                        },
                        "confidence": 0.44515774,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 466,
                              "y": 503
                            },
                            {
                              "x": 455,
                              "y": 503
                            },
                            {
                              "x": 455,
                              "y": 492
                            },
                            {
                              "x": 466,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.36208847,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 452,
                              "y": 503
                            },
                            {
                              "x": 442,
                              "y": 503
                            },
                            {
                              "x": 442,
                              "y": 492
                            },
                            {
                              "x": 452,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.30566624,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 440,
                              "y": 503
                            },
                            {
                              "x": 430,
                              "y": 503
                            },
                            {
                              "x": 430,
                              "y": 492
                            },
                            {
                              "x": 440,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.26879194,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 428,
                              "y": 503
                            },
                            {
                              "x": 418,
                              "y": 503
                            },
                            {
                              "x": 418,
                              "y": 492
                            },
                            {
                              "x": 428,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.2546715,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 403,
                              "y": 503
                            },
                            {
                              "x": 393,
                              "y": 503
                            },
                            {
                              "x": 393,
                              "y": 492
                            },
                            {
                              "x": 403,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.2709255,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 326,
                              "y": 503
                            },
                            {
                              "x": 315,
                              "y": 503
                            },
                            {
                              "x": 315,
                              "y": 492
                            },
                            {
                              "x": 326,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.2530431,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 252,
                              "y": 503
                            },
                            {
                              "x": 241,
                              "y": 503
                            },
                            {
                              "x": 241,
                              "y": 492
                            },
                            {
                              "x": 252,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.27793998,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 188,
                              "y": 503
                            },
                            {
                              "x": 176,
                              "y": 503
                            },
                            {
                              "x": 176,
                              "y": 492
                            },
                            {
                              "x": 188,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.29446027,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 174,
                              "y": 503
                            },
                            {
                              "x": 163,
                              "y": 503
                            },
                            {
                              "x": 163,
                              "y": 492
                            },
                            {
                              "x": 174,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.34936973,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 161,
                              "y": 503
                            },
                            {
                              "x": 149,
                              "y": 503
                            },
                            {
                              "x": 149,
                              "y": 492
                            },
                            {
                              "x": 161,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.28837347,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 148,
                              "y": 503
                            },
                            {
                              "x": 134,
                              "y": 503
                            },
                            {
                              "x": 134,
                              "y": 492
                            },
                            {
                              "x": 148,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.25042403,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 137,
                              "y": 503
                            },
                            {
                              "x": 123,
                              "y": 503
                            },
                            {
                              "x": 123,
                              "y": 492
                            },
                            {
                              "x": 137,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.2917209,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 123,
                              "y": 503
                            },
                            {
                              "x": 112,
                              "y": 503
                            },
                            {
                              "x": 112,
                              "y": 492
                            },
                            {
                              "x": 123,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.47531936,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 113,
                              "y": 503
                            },
                            {
                              "x": 102,
                              "y": 503
                            },
                            {
                              "x": 102,
                              "y": 492
                            },
                            {
                              "x": 113,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.6126359,
                        "property": {
                          "detectedBreak": {
                            "type": "LINE_BREAK"
                          }
                        },
                        "text": "*"
                      }
                    ]
                  }
                ]
              }
            ]
          },
          {
            "blockType": "TEXT",
            "boundingBox": {
              "vertices": [
                {
                  "x": 1365,
                  "y": 208
                },
                {
                  "x": 1412,
                  "y": 208
                },
                {
                  "x": 1412,
                  "y": 241
                },
                {
                  "x": 1365,
                  "y": 241
                }
              ]
            },
            "confidence": 0.99004245,
            "paragraphs": [
              {
                "boundingBox": {
                  "vertices": [
                    {
                      "x": 1365,
                      "y": 208
                    },
                    {
                      "x": 1412,
                      "y": 208
                    },
                    {
                      "x": 1412,
                      "y": 241
                    },
                    {
                      "x": 1365,
                      "y": 241
                    }
                  ]
                },
                "confidence": 0.99004245,
                "words": [
                  {
                    "boundingBox": {
                      "vertices": [
                        {
                          "x": 1365,
                          "y": 208
                        },
                        {
                          "x": 1412,
                          "y": 208
                        },
                        {
                          "x": 1412,
                          "y": 241
                        },
                        {
                          "x": 1365,
                          "y": 241
                        }
                      ]
                    },
                    "confidence": 0.99004245,
                    "symbols": [
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 1365,
                              "y": 209
                            },
                            {
                              "x": 1389,
                              "y": 209
                            },
                            {
                              "x": 1389,
                              "y": 241
                            },
                            {
                              "x": 1365,
                              "y": 241
                            }
                          ]
                        },
                        "confidence": 0.9902449,
                        "text": "1"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 1387,
                              "y": 208
                            },
                            {
                              "x": 1412,
                              "y": 208
                            },
                            {
                              "x": 1412,
                              "y": 240
                            },
                            {
                              "x": 1387,
                              "y": 240
                            }
                          ]
                        },
                        "confidence": 0.98984003,
                        "property": {
                          "detectedBreak": {
                            "type": "LINE_BREAK"
                          }
                        },
                        "text": "7"
                      }
                    ]
                  }
                ]
              }
            ]
          }
        ],
        "confidence": 0.77101094,
        "height": 806,
        "width": 1458
      }
    ],
    "text": "15\n***************\n17"
  },
  "imagePropertiesAnnotation": {
    "cropHints": [
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 510
            },
            {
              "x": 1155
            },
            {
              "x": 1155,
              "y": 805
            },
            {
              "x": 510,
              "y": 805
            }
          ]
        },
        "confidence": 0.5625001,
        "importanceFraction": 0.67390424
      },
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 291
            },
            {
              "x": 1097
            },
            {
              "x": 1097,
              "y": 805
            },
            {
              "x": 291,
              "y": 805
            }
          ]
        },
        "confidence": 0.5625,
        "importanceFraction": 0.84238034
      },
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 218
            },
            {
              "x": 1185
            },
            {
              "x": 1185,
              "y": 805
            },
            {
              "x": 218,
              "y": 805
            }
          ]
        },
        "confidence": 0.55645883,
        "importanceFraction": 1
      }
    ],
    "dominantColors": {
      "colors": [
        {
          "color": {
            "blue": 245,
            "green": 245,
            "red": 245
          },
          "hex": "F5F5F5",
          "percent": 28.497889999999998,
          "percentRounded": 28,
          "pixelFraction": 0.8969478,
          "rgb": "245, 245,\n 245",
          "score": 0.2849789
        },
        {
          "color": {
            "blue": 119,
            "green": 119,
            "red": 119
          },
          "hex": "777777",
          "percent": 18.050036,
          "percentRounded": 18,
          "pixelFraction": 0.027630523,
          "rgb": "119, 119,\n 119",
          "score": 0.18050036
        },
        {
          "color": {
            "blue": 19,
            "green": 19,
            "red": 19
          },
          "hex": "131313",
          "percent": 7.161156,
          "percentRounded": 7,
          "pixelFraction": 0.004176707,
          "rgb": "19, 19,\n 19",
          "score": 0.07161156
        },
        {
          "color": {
            "blue": 153,
            "green": 153,
            "red": 153
          },
          "hex": "999999",
          "percent": 15.615201,
          "percentRounded": 16,
          "pixelFraction": 0.02562249,
          "rgb": "153, 153,\n 153",
          "score": 0.15615201
        },
        {
          "color": {
            "blue": 87,
            "green": 87,
            "red": 87
          },
          "hex": "575757",
          "percent": 13.579108000000002,
          "percentRounded": 14,
          "pixelFraction": 0.019678716,
          "rgb": "87, 87,\n 87",
          "score": 0.13579108
        },
        {
          "color": {
            "blue": 190,
            "green": 190,
            "red": 190
          },
          "hex": "BEBEBE",
          "percent": 11.086322,
          "percentRounded": 11,
          "pixelFraction": 0.021606425,
          "rgb": "190, 190,\n 190",
          "score": 0.11086322
        },
        {
          "color": {
            "blue": 51,
            "green": 51,
            "red": 51
          },
          "hex": "333333",
          "percent": 6.010287,
          "percentRounded": 6,
          "pixelFraction": 0.0043373494,
          "rgb": "51, 51,\n 51",
          "score": 0.06010287
        }
      ]
    }
  },
  "labelAnnotations": [
    {
      "description": "Automotive lighting",
      "mid": "/m/0768fx",
      "score": 0.92270863,
      "topicality": 0.92270863
    },
    {
      "description": "Gesture",
      "mid": "/m/0244x1",
      "score": 0.85260487,
      "topicality": 0.85260487
    },
    {
      "description": "Automotive design",
      "mid": "/m/068mqj",
      "score": 0.8496454,
      "topicality": 0.8496454
    },
    {
      "description": "Font",
      "mid": "/m/03gq5hm",
      "score": 0.7689706,
      "topicality": 0.7689706
    },
    {
      "description": "Automotive exterior",
      "mid": "/m/0h8ls87",
      "score": 0.75901467,
      "topicality": 0.75901467
    },
    {
      "description": "Auto part",
      "mid": "/m/08dz3q",
      "score": 0.7510306,
      "topicality": 0.7510306
    },
    {
      "description": "Coloring book",
      "mid": "/m/05h7rm",
      "score": 0.7148846,
      "topicality": 0.7148846
    },
    {
      "description": "Drawing",
      "mid": "/m/02csf",
      "score": 0.71441007,
      "topicality": 0.71441007
    },
    {
      "description": "Circle",
      "mid": "/m/01vkl",
      "score": 0.71318007,
      "topicality": 0.71318007
    },
    {
      "description": "Elbow",
      "mid": "/m/01vm1p",
      "score": 0.71267515,
      "topicality": 0.71267515
    },
    {
      "description": "Diagram",
      "mid": "/m/02v0m2",
      "score": 0.709528,
      "topicality": 0.709528
    },
    {
      "description": "Fashion accessory",
      "mid": "/m/0463sg",
      "score": 0.67354536,
      "topicality": 0.67354536
    },
    {
      "description": "Line art",
      "mid": "/m/0919rx",
      "score": 0.65924,
      "topicality": 0.65924
    },
    {
      "description": "Illustration",
      "mid": "/m/01kr8f",
      "score": 0.63600135,
      "topicality": 0.63600135
    },
    {
      "description": "Art",
      "mid": "/m/0jjw",
      "score": 0.63068235,
      "topicality": 0.63068235
    },
    {
      "description": "Automotive wheel system",
      "mid": "/m/0h8ly30",
      "score": 0.59069836,
      "topicality": 0.59069836
    },
    {
      "description": "Symbol",
      "mid": "/m/09ddf",
      "score": 0.58919644,
      "topicality": 0.58919644
    },
    {
      "description": "Slope",
      "mid": "/m/078hm",
      "score": 0.5772801,
      "topicality": 0.5772801
    },
    {
      "description": "Light bulb",
      "mid": "/m/0h8l4fh",
      "score": 0.5681215,
      "topicality": 0.5681215
    },
    {
      "description": "Triangle",
      "mid": "/m/07jx7",
      "score": 0.56783974,
      "topicality": 0.56783974
    },
    {
      "description": "Oval",
      "mid": "/m/06g58b",
      "score": 0.56316096,
      "topicality": 0.56316096
    },
    {
      "description": "Sketch",
      "mid": "/m/07glzq",
      "score": 0.5614814,
      "topicality": 0.5614814
    },
    {
      "description": "Monochrome",
      "mid": "/m/01mwkf",
      "score": 0.5512363,
      "topicality": 0.5512363
    },
    {
      "description": "Artwork",
      "mid": "/m/0dgsmq8",
      "score": 0.5144047,
      "topicality": 0.5144047
    },
    {
      "description": "Logo",
      "mid": "/m/0dwx7",
      "score": 0.51423424,
      "topicality": 0.51423424
    }
  ],
  "localizedObjectAnnotations": [
    {
      "boundingPoly": {
        "normalizedVertices": [
          {
            "x": 0.37679967,
            "y": 0.066629104
          },
          {
            "x": 0.91358244,
            "y": 0.066629104
          },
          {
            "x": 0.91358244,
            "y": 0.9314933
          },
          {
            "x": 0.37679967,
            "y": 0.9314933
          }
        ]
      },
      "mid": "/m/0hf58v5",
      "name": "Luggage & bags",
      "score": 0.6067607
    }
  ],
  "logoAnnotations": [
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 243,
            "y": 15
          },
          {
            "x": 1415,
            "y": 15
          },
          {
            "x": 1415,
            "y": 806
          },
          {
            "x": 243,
            "y": 806
          }
        ]
      },
      "description": "Alienware",
      "mid": "/m/042kgg",
      "score": 0.6777873
    }
  ],
  "safeSearchAnnotation": {
    "adult": "VERY_UNLIKELY",
    "medical": "VERY_UNLIKELY",
    "racy": "UNLIKELY",
    "spoof": "UNLIKELY",
    "violence": "VERY_UNLIKELY"
  },
  "textAnnotations": [
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 44,
            "y": 208
          },
          {
            "x": 1412,
            "y": 208
          },
          {
            "x": 1412,
            "y": 522
          },
          {
            "x": 44,
            "y": 522
          }
        ]
      },
      "description": "15\n***************\n17",
      "locale": "und"
    },
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 44,
            "y": 493
          },
          {
            "x": 86,
            "y": 493
          },
          {
            "x": 86,
            "y": 522
          },
          {
            "x": 44,
            "y": 522
          }
        ]
      },
      "description": "15"
    },
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 475,
            "y": 504
          },
          {
            "x": 102,
            "y": 504
          },
          {
            "x": 102,
            "y": 492
          },
          {
            "x": 475,
            "y": 492
          }
        ]
      },
      "description": "***************"
    },
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 1365,
            "y": 208
          },
          {
            "x": 1412,
            "y": 208
          },
          {
            "x": 1412,
            "y": 241
          },
          {
            "x": 1365,
            "y": 241
          }
        ]
      },
      "description": "17"
    }
  ]
}